In [19]:
import pandas as pd 
import numpy as np
from sklearn import tree

In [137]:
data_train = pd.read_csv('train.csv')
data_test = pd.read_csv('test.csv')
gender =pd.read_csv('gender_submission.csv')
data_train.tail()


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.00,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.00,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.45,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.00,C148,C
890,891,0,3,"Dooley, Mr. Patrick",male,32.0,0,0,370376,7.75,NaN,Q


In [138]:
data_train['Sex'].replace({'male':1,'female':0},inplace=True)
data_test['Sex'].replace({'male':1,'female':0},inplace=True)
m = round(data_train['Age'].mean(),0)
n = round(data_test['Age'].mean(),0)
avg = (n+m)/2.
data_train['Age'].fillna(avg,inplace=True)
data_test['Age'].fillna(avg,inplace=True)


In [139]:
data_train.Fare=data_train.Fare.replace(0,np.NAN)
print(data_train.groupby(data_train.Pclass).Fare.mean())
print(data_test.groupby(data_test.Pclass).Fare.mean())
means=[] 
for i in range (1,4):
    means.append((data_train.groupby(data_train.Pclass).Fare.mean()[i]+data_test.groupby(data_test.Pclass).Fare.mean()[i])/2)
print(means)    

Pclass
1    86.148874
2    21.358661
3    13.787875
Name: Fare, dtype: float64
Pclass
1    94.280297
2    22.202104
3    12.459678
Name: Fare, dtype: float64
[90.21458580192228, 21.78038276851516, 13.123776414424817]


In [140]:
train_p1=pd.DataFrame(data_train[data_train.Pclass==1])
train_p2=pd.DataFrame(data_train[data_train.Pclass==2])
train_p3=pd.DataFrame(data_train[data_train.Pclass==3])
train_p1.Fare.fillna(means[0],inplace=True)
train_p2.Fare.fillna(means[1],inplace=True)
train_p3.Fare.fillna(means[2],inplace=True)
data_train=pd.concat([data_train_p1,data_train_p2,data_train_p3])
data_train.sort_index(inplace=True)

In [141]:
test_p1=pd.DataFrame(data_test[data_test.Pclass==1])
test_p2=pd.DataFrame(data_test[data_test.Pclass==2])
test_p3=pd.DataFrame(data_test[data_test.Pclass==3])
test_p1.Fare.fillna(means[0],inplace=True)
test_p2.Fare.fillna(means[1],inplace=True)
test_p3.Fare.fillna(means[2],inplace=True)
data_test=pd.concat([data_test_p1,data_test_p2,data_test_p3])
data_test.sort_index(inplace=True)

In [142]:
data_train.loc[data_train['Cabin'].notnull(), 'Cabin'] = 1
data_train['Cabin'].fillna(0,inplace=True)


data_train.tail()


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
886,887,0,2,"Montvila, Rev. Juozas",1,27.0,0,0,211536,13.00,0,S
887,888,1,1,"Graham, Miss. Margaret Edith",0,19.0,0,0,112053,30.00,1,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",0,30.0,1,2,W./C. 6607,23.45,0,S
889,890,1,1,"Behr, Mr. Karl Howell",1,26.0,0,0,111369,30.00,1,C
890,891,0,3,"Dooley, Mr. Patrick",1,32.0,0,0,370376,7.75,0,Q


In [143]:
data_test.loc[data_test['Cabin'].notnull(), 'Cabin'] = 1
data_test['Cabin'].fillna(0,inplace=True)


data_test.tail()


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
413,1305,3,"Spector, Mr. Woolf",1,30.0,0,0,A.5. 3236,8.0500,0,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",0,39.0,0,0,PC 17758,108.9000,1,C
415,1307,3,"Saether, Mr. Simon Sivertsen",1,38.5,0,0,SOTON/O.Q. 3101262,7.2500,0,S
416,1308,3,"Ware, Mr. Frederick",1,30.0,0,0,359309,8.0500,0,S
417,1309,3,"Peter, Master. Michael J",1,30.0,1,1,2668,22.3583,0,C


In [144]:
features=['Pclass','Sex','SibSp','Parch','Cabin','Age','Fare']
X=data_train[features]
Y=data_train['Survived']

In [145]:
>>> from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier 

rf=RandomForestClassifier(n_estimators=100, max_depth=5, random_state=1)


In [148]:
rf.fit(X,Y)
print(cross_val_score(rf, X, Y, cv=100).mean())


0.8219444444444445


In [149]:
predictions = rf.predict(data_test[features])


In [150]:
output = pd.DataFrame({'PassengerId': data_test.PassengerId, 'Survived': predictions})
output.to_csv('my_submission.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!
